In [6]:
# Start by importing the relevant libraries.
import pandas as pd
import pickle
import numpy as np

In [3]:
# load delivery dataframe from dataset exported under data_extraction
df = pickle.load(open('dataset_level2.pkl', 'rb'))

From here on, we will work on modifying this DataFrame to contain data for the 8 features we want to use for the developement of our model. These features are as follows:
-> batting team
-> bowling team
-> current score
-> wickets left
-> current run rate
-> city
-> balls left
-> last five overs (runs scored)

In [5]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,1,Australia,Sri Lanka,0.1,0,0,NaN,Melbourne Cricket Ground
1,1,Australia,Sri Lanka,0.2,0,0,NaN,Melbourne Cricket Ground
2,1,Australia,Sri Lanka,0.3,1,0,NaN,Melbourne Cricket Ground
3,1,Australia,Sri Lanka,0.4,2,0,NaN,Melbourne Cricket Ground
4,1,Australia,Sri Lanka,0.5,0,0,NaN,Melbourne Cricket Ground
...,...,...,...,...,...,...,...,...
115320,963,Sri Lanka,Australia,19.3,1,0,Colombo,R Premadasa Stadium
115321,963,Sri Lanka,Australia,19.4,0,0,Colombo,R Premadasa Stadium
115322,963,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,R Premadasa Stadium
115323,963,Sri Lanka,Australia,19.6,2,0,Colombo,R Premadasa Stadium


In [8]:
# Start by focusing on the city column.
# Count the number of NaN values in the city column.
df['city'].isnull().sum()

8548

In [12]:
# For the line items with NaN city values, look at the venue values.
df[df['city'].isnull()]['venue'].value_counts()

venue
Dubai International Cricket Stadium        2969
Pallekele International Cricket Stadium    2066
Melbourne Cricket Ground                   1453
Sydney Cricket Ground                       749
Adelaide Oval                               498
Harare Sports Club                          372
Sharjah Cricket Stadium                     249
Sylhet International Cricket Stadium        128
Carrara Oval                                 64
Name: count, dtype: int64

In [14]:
# The output above indicates that for NaN values in the city column, we can use the first word from the values 
# of the venue column.
# We create a condition using np.where() to assign values where city column value is NaN
# These values are outputted to an array called cities that can then update the DataFrame.
cities = np.where(df['city'].isnull(), df['venue'].str.split().apply(lambda x:x[0]), df['city'])
df['city'] = cities

In [15]:
# Check all columns for any NaN values.
df.isnull().sum()

match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
city                0
venue               0
dtype: int64

In [16]:
# Remove the Venue column since it is not needed anymore = not a feature, redundant with city.
df.drop(columns = ['venue'], inplace = True)

In [17]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,1,Australia,Sri Lanka,0.1,0,0,Melbourne
1,1,Australia,Sri Lanka,0.2,0,0,Melbourne
2,1,Australia,Sri Lanka,0.3,1,0,Melbourne
3,1,Australia,Sri Lanka,0.4,2,0,Melbourne
4,1,Australia,Sri Lanka,0.5,0,0,Melbourne
...,...,...,...,...,...,...,...
115320,963,Sri Lanka,Australia,19.3,1,0,Colombo
115321,963,Sri Lanka,Australia,19.4,0,0,Colombo
115322,963,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo
115323,963,Sri Lanka,Australia,19.6,2,0,Colombo
